In [0]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)

In [0]:
df = pd.read_csv("https://raw.githubusercontent.com/JackyP/testing/master/datasets/nycflights.csv", index_col=0)
df["data"] = pd.to_datetime(df[['year', 'month', 'day']]) 

# Coluna que indica atraso ou não no voo
df['isdelay'] = (df['arr_delay'] > 0).astype(int)

#Coluna com as estações
def get_season(date):
    month = date.month
    if month in [12, 1, 2]:
        return 'Winter'
    elif month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    else:
        return 'Autumn'

df['season'] = df['data'].apply(get_season)

# Coluna com a rota (origem + destino)
df['route'] = df['origin'] + '-' + df['dest']


In [0]:
summary = {
    'total_flyies': len(df),
    'delay': (df['isdelay'] == 1).sum(),
    'no_delay': (df['isdelay'] == 0).sum()
}
display(pd.DataFrame([summary]))

**Conclusão quantitativa:**

1. Total de voos é igual a 336.8k
2. Total de atrasos é de 133k.
3. Quase 40% dos voos atrasam para chegar ao destino.

### Listando as 10 companias que mais atrasam voos e sumarizando as demais.
### 



In [0]:
carrier_delay_counts = df.groupby('carrier')['isdelay'].sum().reset_index()
carrier_delay_counts = carrier_delay_counts.sort_values(by='isdelay', ascending=False)

top10 = carrier_delay_counts.head(10)
rest_sum = carrier_delay_counts.iloc[10:]['isdelay'].sum()
rest_row = pd.DataFrame({'carrier': ['Others'], 'isdelay': [rest_sum]})

plot_data = pd.concat([top10, rest_row], ignore_index=True)

# Divide by 1000, round to 1 decimal, and add 'K'
plot_data['isdelay_k'] = (plot_data['isdelay'] / 1000).round(1).astype(str) + 'K'

sns.set(style="whitegrid")
plt.figure(figsize=(10,6))
ax = sns.barplot(x='carrier', y='isdelay', data=plot_data)
plt.xlabel('Carrier')
plt.ylabel('')  # Remove y-axis label
plt.title('Top 10 Airlines by Total Delays (Others Summarized)')
plt.xticks(rotation=0)
plt.tight_layout()

# Remove y-axis ticks and values
ax.yaxis.set_ticks([])
ax.yaxis.set_ticklabels([])

# Add value labels on top of each bar (in 'K' format)
for i, p in enumerate(ax.patches):
    ax.annotate(plot_data['isdelay_k'].iloc[i], 
                (p.get_x() + p.get_width() / 2, p.get_height()), 
                ha='center', va='bottom', fontsize=10)

plt.show()

**Conclusão Quantitativa do gráfico**

1) Ao todo encontramos 133k voos atrasados e as 10 mais totalizam 98% dos atrasos, restando 2.7K casos de atrasos para as demais companias.
2) As comanias EV, B6 e UA são as campeãs de atraso, e sozinhas superam 50% dos atrasos. Mais de 70K casos dos 133k.



### Listando as 10 companias que mais atrasam, mas comparando com seu próprio total de voos ###


In [0]:
# Filtra as 10 empresas com mais isdelay = 1
top10_carriers = df[df['isdelay'] == 1].groupby('carrier').size().sort_values(ascending=False).head(10).index

# Filtra o DataFrame para apenas essas empresas
filtered_df = df[df['carrier'].isin(top10_carriers)]

stacked_data = filtered_df.groupby(['carrier', 'isdelay']).size().unstack(fill_value=0)
# Ordena de forma decrescente pelo número de atrasos (isdelay=1)
stacked_data = stacked_data.sort_values(by=1, ascending=False)

plt.figure(figsize=(14,7))
ax = stacked_data.plot(kind='bar', stacked=True, colormap='tab20', ax=plt.gca())
plt.xlabel('Carrier')
plt.ylabel('')  # Remove y-axis label
plt.title('Top 10 Airlines with the most delays')
plt.legend(title='', labels=['No Delay', 'Delay'])
plt.tight_layout()

# Adiciona percentual de atrasos dentro da barra de isdelay=1
for i, carrier in enumerate(stacked_data.index):
    total = stacked_data.loc[carrier].sum()
    delay = stacked_data.loc[carrier, 1]
    percent = (delay / total) * 100 if total > 0 else 0
    bar = ax.patches[i + len(stacked_data)]  # patches: first half is isdelay=0, second half is isdelay=1
    ax.text(
        bar.get_x() + bar.get_width() / 2,
        bar.get_y() + bar.get_height() / 2,
        f'{percent:.1f}%',
        ha='center', va='center', color='white', fontsize=11, fontweight='bold'
    )
    # Adiciona o total de voos no topo de cada barra, dividido por 1000, 1 dígito após o ponto e "K"
    total_k = f'{(total/1000):.1f}K'
    ax.text(
        bar.get_x() + bar.get_width() / 2,
        bar.get_y() + bar.get_height(),
        total_k,
        ha='center', va='bottom', color='black', fontsize=12, fontweight='bold'
    )

# Remove y-axis ticks and values
ax.yaxis.set_ticks([])
ax.yaxis.set_ticklabels([])

plt.show()

**Conclusões Quantidativas do gráfico:**

1) Na lista das top 10, que mais atrasam, todas superam a marca de 34% de seus voos.


**Conclusões qualitativas dos gráficos:**

1. As 10 companhias aéreas com maior número absoluto de atrasos concentram a maior parte dos voos atrasados, com as demais ('Others') representando uma fração menor do total.
2. Entre as 10 companhias com mais atrasos, algumas apresentam uma alta proporção de voos atrasados em relação ao seu próprio total de voos, indicando que o problema de atraso é mais recorrente em certas empresas, caso da compania FL que atrasa mais de 58% dos seu baixo número de voos.
3. A análise conjunta mostra que nem sempre a companhia com mais voos atrasados em números absolutos é a que tem maior percentual de atrasos, sugerindo que o volume de operações influencia, mas não determina a eficiência operacional.


# FIM DA ANÁLISE SOBRE QUESTÃO 01


### Analisando o comportamento dos 50 destinos que mais registram atrasos



In [0]:
df_dest = df[df['isdelay'] == 1].groupby('dest').size()
df_dest = df_dest.sort_values(ascending=False)
top50 = df_dest.head(50)
other = df_dest.iloc[50:].sum()
df_dest_top = top50.copy()
df_dest_top['Other'] = other

ax = df_dest_top.plot(
    kind='bar',
    figsize=(12,6),
    color=sns.color_palette("Set2", 1)
)
plt.xlabel('Destino')
plt.ylabel('Quantidade de voos atrasados')
plt.title('Quantidade de voos atrasados por destino (Top 50 + Outros)')
plt.legend(['Atrasado'])
plt.tight_layout()
plt.show()

### Analisando o comportamento por rotas que chegaram ao destino ATL, que foi o que mais registro atrasos


In [0]:
# Total distinto de Rotas
distinct_route_count = df['route'].nunique()
display(pd.DataFrame({'distinct_route_count': [distinct_route_count]}))


In [0]:
#Considerando que temos 224 rotas distintas, 219 registraram algum tipo de atraso. 

route_count_by_isdelay = df[df['isdelay'] == 1].groupby('isdelay')['route'].nunique().reset_index()
display(route_count_by_isdelay)

In [0]:
# Calculando a média, mediana e moda dos percentual de atraso.

from scipy import stats

mean_delay = grouped['delay_percent'].mean()
median_delay = grouped['delay_percent'].median()
mode_delay = stats.mode(grouped['delay_percent'], keepdims=True)[0][0]

display(pd.DataFrame({
    'mean_delay_percent': [mean_delay],
    'median_delay_percent': [median_delay],
    'mode_delay_percent': [mode_delay]
}))

In [0]:
#Analisando apenas as rotas que compoem o destino ATL, o que mais registrou atrasos.

grouped = df[df['dest'] == 'ATL'].groupby('route')['isdelay'].agg(
    delay=lambda x: (x == 1).sum(),
    no_delay=lambda x: (x == 0).sum(),
    total='count'
).reset_index()

grouped['delay_percent'] = (grouped['delay'] / grouped['total']) * 100

grouped = grouped.sort_values(by='delay', ascending=False)

grouped.head(50)

**Conclusão parcial da Questão 02**

1. Se 219 rotas das 224 possíveis já registraram algum tipo de atraso.
2. A Média e Mediana de atrasos são de aproximadamente 40%. Percentual este comprovado com as 3 rotas que chegam o destino ATL. LGA, EWR E JFK.

Então, não podemos afirmar que as rotas influênciam no atraso.



In [0]:
#Total de aeronaves

distinct_tailnum_count = df['tailnum'].nunique()
display(pd.DataFrame({'distinct_tailnum_count': [distinct_tailnum_count]}))

In [0]:
#media, mediana e moda do número de voos por aeronave

from scipy import stats

tailnum_counts = df['tailnum'].value_counts()

mean_tailnum = tailnum_counts.mean()
median_tailnum = tailnum_counts.median()
mode_tailnum = stats.mode(tailnum_counts, keepdims=True)[0][0]

display(pd.DataFrame({
    'mean_tailnum_flights': [mean_tailnum],
    'median_tailnum_flights': [median_tailnum],
    'mode_tailnum_flights': [mode_tailnum]
}))

In [0]:
# Grupos de aeronaves por isdelay
tailnum_delay = set(df[df['isdelay'] == 1]['tailnum'].unique())
tailnum_no_delay = set(df[df['isdelay'] == 0]['tailnum'].unique())

# Aeronaves que aparecem apenas no grupo isdelay = 0
exclusive_no_delay = tailnum_no_delay - tailnum_delay

# Filtrando e contando o número de voos por tailnum
df_exclusive = df[df['tailnum'].isin(exclusive_no_delay)]
flight_counts = df_exclusive.groupby('tailnum').size().reset_index(name='flight_count')
flight_counts = flight_counts.sort_values(by='flight_count', ascending=False)

flight_counts.head(5)

In [0]:
# Seleciona tailnum exclusivos sem atraso
exclusive_no_delay = set(df[df['isdelay'] == 0]['tailnum'].unique()) - set(df[df['isdelay'] == 1]['tailnum'].unique())

# Filtra o DataFrame para esses tailnum e conta o total de voos
total_flights_exclusive_no_delay = df[df['tailnum'].isin(exclusive_no_delay)].shape[0]

display(pd.DataFrame({'total_flights_exclusive_no_delay': [total_flights_exclusive_no_delay]}))

**Conclusão sobre as aeronaves:**

1. Foram encontradas 4043 aeronaves distintas, que totalizaram 336.776 voos.
2. Destas 4K apenas 170 não tiveram nenhum atraso em todo o período apurado, mas este pequeno grupo só realizou 2839 voos, e isso equivale a 0,8% do total.

Então, é possível afirmar que as aeronaves também não influênciam no atraso, pois do pequeno grupo que não causou nenhum tipo de atraso, eles totalizam menos de 1% do total de voos.


### Resultado final da Questão 02

Nem aeronaves ou rotas influênciam nos atrasos, pois os poucos casos que não tiveram nenhum tipo de atraso, possuem valores muito baixo para contribuierem com a analise.


# FIM DA ANÁLISE SOBRE QUESTÃO 02

### Apurando os períodos que mais influenciam nos atrasos

Como já sabemos que aeronaves e rotas não influênciam, decidimos investigar padrões por períodos.

In [0]:
df['month_str'] = df['data'].dt.strftime('%b')
df_monthly = df.groupby(['month_str', 'isdelay']).size().unstack(fill_value=0)

df_monthly = df_monthly.reindex(['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'])

# Invert columns so delayed flights are at the bottom
df_monthly = df_monthly[[1, 0]]

ax = df_monthly.plot(
    kind='bar',
    stacked=True,
    figsize=(10,6),
    color=sns.color_palette("Set2", 2)
)
plt.xlabel('Month')
plt.ylabel('')
plt.title('Number of Flights per Month (Delayed and Not Delayed)')
plt.legend(['Delayed', 'Not Delayed'], loc='center left', bbox_to_anchor=(1, 0.5))
plt.tight_layout()

# Add total flights on top of each column, divided by 1000 and with "K"
for idx, total in enumerate(df_monthly.sum(axis=1)):
    ax.text(
        idx, 
        total + 50,  # 50 for spacing above the bar
        f"{total/1000:.1f}K",
        ha='center',
        va='bottom',
        fontsize=10,
        fontweight='bold'
    )

# Add delay percentage inside the delayed bar
for idx, (delayed, total) in enumerate(zip(df_monthly[1], df_monthly.sum(axis=1))):
    percent = (delayed / total * 100) if total > 0 else 0
    ax.text(
        idx,
        delayed / 2,
        f"{percent:.1f}%",
        ha='center',
        va='center',
        color='black',
        fontsize=10,
        fontweight='bold'
    )

ax.yaxis.set_ticks([])  # Remove Y axis values
plt.show()

In [0]:
df_season_delay = df.groupby(['season', 'isdelay']).size().unstack(fill_value=0)
df_season_delay['% delay'] = (df_season_delay[1] / df_season_delay.sum(axis=1) * 100).round(2)

df_season_delay.plot(
    kind='bar',
    stacked=True,
    figsize=(10,6),
    color=sns.color_palette("Set2", 2)
)
for idx, row in enumerate(df_season_delay['% delay']):
    plt.text(
        idx, 
        df_season_delay.loc[df_season_delay.index[idx], 0] + df_season_delay.loc[df_season_delay.index[idx], 1] + 2,
        f"{row}%",
        ha='center',
        fontsize=10
    )
plt.xlabel('Season')
plt.ylabel('Number of flights')
plt.title('Number of flights per season (Delayed and Not Delayed)')
plt.legend(['Not Delayed', 'Delayed'])
plt.tight_layout()
plt.show()

In [0]:
filtered_carriers = ['EV', 'B6', 'UA', 'DL', 'MQ']
df_filtered = df[df['carrier'].isin(filtered_carriers)]

top15_dest = df_filtered[df_filtered['isdelay'] == 1].groupby('dest').size().sort_values(ascending=False).head(15).index.tolist()
df_top15 = df_filtered[df_filtered['dest'].isin(top15_dest)]

heatmap_data = df_top15.groupby(['dest', 'season'])['isdelay'].sum().unstack(fill_value=0)

plt.figure(figsize=(10, 6))
sns.heatmap(heatmap_data, annot=True, fmt='d', cmap='YlOrRd')
plt.title('Heatmap of Delays by Destination and Season (Top 15 Destinations) - Carriers EV, B6, UA, DL, MQ')
plt.xlabel('Season')
plt.ylabel('Destination')
plt.tight_layout()
plt.show()

**Conclusão sobre padrões ou tendências nos atrasos:**

Como já haviamos investigado que aeronaves e rotas não tinham relevância ou influenciavam os atrasos, decidimos avaliar por período.

1. Descobrimos que alguns meses possuiam mais atrasos que outros, como Dezembro e Julho.
2. Então resolvemos agrupar um pouco mais e classificamos por estações do ano, e foi possível perceber que durante Verão e Inverno os atrasos superam os 43% e ao longo da questão 2, apuramos que tanto média quando mediana dos atrasos eram de 39%.
3. Para completar o heatmap entre os destinos que mais atrasam pelas estações do ano, foi possivel constatar um maior número de atrasos nestes destinos, durante as estações Inverno e Verão.

Então, podemos afirmar que durante o Inverno e o Verão, os números de voos atrasados tendem a crescer em comparação com as demais estações.
